In [1]:
import tensorflow as tf
import pandas as pd
import tqdm
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import math

In [2]:
df= pd.read_csv("../Data/train.csv")
all_id = np.array(df['Id'])
y= np.array(df['Pawpularity'])


In [3]:
dict_images = {}
img_size=128
for i, single_id in tqdm.tqdm(enumerate(all_id)):
    img = cv2.cvtColor(cv2.imread(f"../Data/train/{single_id}.jpg"), cv2.COLOR_BGR2RGB)

    if img.shape[0] > img.shape[1]:
        bigger = 0
        smaller = 1
    else:
        bigger = 1
        smaller =0
    new_size = img.shape[smaller]
    to_cut_out = img.shape[bigger] - new_size
    if img.shape[0] > img.shape[1]:
        new_img = img[int(to_cut_out/2):new_size+int(to_cut_out/2)]
    else:
        new_img = img[:,int(to_cut_out/2):new_size+int(to_cut_out/2)]
    new_img = cv2.resize(new_img, (img_size, img_size))
    dict_images[single_id] = new_img/255


9912it [01:22, 119.99it/s]


In [4]:
def get_images_from_id(data_id, img_size=128, colours=3):
    pictures = np.zeros((len(data_id), img_size, img_size, colours))
    for i, single_id in enumerate(data_id):
        pictures[i] = dict_images[single_id]
    return pictures

In [5]:
def get_model():
    tf.random.set_seed(1234)
    IMG_SIZE = 128
    METADATA_SHAPE = 12
    input_image = tf.keras.Input((IMG_SIZE, IMG_SIZE, 3)) # Wejście na zdjęcia 128X128x3
    input_metadata = tf.keras.Input(METADATA_SHAPE) # Wejście na metadane 12 kategorii

    image_conv2D_1 = tf.keras.layers.Conv2D(8, 3, activation='relu', padding='same')(input_image)
    image_conv2D_2 = tf.keras.layers.Conv2D(8, 3, activation='relu', padding='same')(image_conv2D_1)
    image_maxpool2D_1 = tf.keras.layers.MaxPooling2D(2)(image_conv2D_2)

    image_conv2D_3 = tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same')(image_maxpool2D_1)
    image_conv2D_4 = tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same')(image_conv2D_3)
    image_maxpool2D_2 = tf.keras.layers.MaxPooling2D(2)(image_conv2D_4)

    image_conv2D_5 = tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same')(image_maxpool2D_2)
    image_conv2D_6 = tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same')(image_conv2D_5)
    image_maxpool2D_3 = tf.keras.layers.MaxPooling2D(2)(image_conv2D_6)

    image_conv2D_7 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(image_maxpool2D_3)
    image_conv2D_8 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(image_conv2D_7)
    image_maxpool2D_4 = tf.keras.layers.MaxPooling2D(2)(image_conv2D_8)

    image_conv2D_9 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same')(image_maxpool2D_4)
    image_conv2D_10 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same')(image_conv2D_9)
    image_maxpool2D_5 = tf.keras.layers.MaxPooling2D(2)(image_conv2D_10) 
    flatten = tf.keras.layers.GlobalAveragePooling2D()(image_maxpool2D_5) #Podobne działanie do Flatten

    metadata_dense_1 = tf.keras.layers.Dense(16, activation="relu", kernel_regularizer=tf.keras.regularizers.l2())(input_metadata)

    concat = tf.keras.layers.concatenate([flatten, metadata_dense_1])
    output = tf.keras.layers.Dense(1)(concat)
    model = tf.keras.Model(inputs=[input_image, input_metadata], outputs=[output])

    model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam(1e-3), metrics=["mse"])
    return model

In [6]:
times_cross_validation = 5
kf = KFold(n_splits=times_cross_validation, shuffle=True, random_state=1234)
results = np.zeros(5)

In [7]:
def get_metadata_and_photos(selected_id):
    X_metadata = df[df['Id'].isin(selected_id)]
    X_pictures = get_images_from_id(X_metadata['Id'])
    X_metadata = X_metadata.drop(columns=['Id', 'Pawpularity'])
    return X_metadata, X_pictures

In [8]:
np.random.seed(1410)
tf.random.set_seed(1234)
for i, (train_index, test_index) in tqdm.tqdm(enumerate(kf.split(all_id))):
    X_train_full_id = all_id[train_index]
    X_test_id = all_id[test_index]
    y_train_full = y[train_index]
    y_test = y[test_index]
    
    X_train_id, X_valid_id, y_train, y_valid = train_test_split(X_train_full_id, y_train_full, test_size=0.2, random_state=42)
    
    X_train_metadata, X_train_pictures = get_metadata_and_photos(X_train_id)
    X_valid_metadata, X_valid_pictures = get_metadata_and_photos(X_valid_id)
    X_test_metadata, X_test_pictures = get_metadata_and_photos(X_test_id)
    
    model = get_model()
    
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True)
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
    model.fit((X_train_pictures, X_train_metadata), y_train, epochs=300, validation_data=((X_valid_pictures, X_valid_metadata), y_valid), callbacks=[checkpoint_cb, early_stopping_cb])
    model = tf.keras.models.load_model('best_model.h5')
    y_pred = model.predict((X_test_pictures, X_test_metadata))
    results[i] = math.sqrt(mean_squared_error(y_test, y_pred))

0it [00:00, ?it/s]

Train on 6343 samples, validate on 1586 samples
Epoch 1/300
6343/6343 [==============================] - 11s 2ms/sample - loss: 525.5029 - mse: 525.3713 - val_loss: 444.2533 - val_mse: 444.1286
Epoch 2/300
6343/6343 [==============================] - 6s 1ms/sample - loss: 450.4497 - mse: 450.3291 - val_loss: 453.0493 - val_mse: 452.9325
Epoch 3/300
6343/6343 [==============================] - 7s 1ms/sample - loss: 437.4287 - mse: 437.3144 - val_loss: 428.3789 - val_mse: 428.2669
Epoch 4/300
6343/6343 [==============================] - 7s 1ms/sample - loss: 442.7796 - mse: 442.6691 - val_loss: 429.1303 - val_mse: 429.0210
Epoch 5/300
6343/6343 [==============================] - 7s 1ms/sample - loss: 434.1909 - mse: 434.0822 - val_loss: 431.6035 - val_mse: 431.4962
Epoch 6/300
6343/6343 [==============================] - 6s 1ms/sample - loss: 436.3250 - mse: 436.2178 - val_loss: 441.4839 - val_mse: 441.3773
Epoch 7/300
6343/6343 [==============================] - 7s 1ms/sample - loss: 43

1it [02:45, 165.32s/it]

Train on 6343 samples, validate on 1586 samples
Epoch 1/300
6343/6343 [==============================] - 8s 1ms/sample - loss: 517.6422 - mse: 517.5107 - val_loss: 441.3251 - val_mse: 441.2007: 5
Epoch 2/300
6343/6343 [==============================] - 7s 1ms/sample - loss: 452.7247 - mse: 452.6049 - val_loss: 427.9016 - val_mse: 427.7858
Epoch 3/300
6343/6343 [==============================] - 6s 1ms/sample - loss: 452.7639 - mse: 452.6506 - val_loss: 438.7140 - val_mse: 438.6030
Epoch 4/300
6343/6343 [==============================] - 6s 1ms/sample - loss: 439.6435 - mse: 439.5346 - val_loss: 429.2347 - val_mse: 429.1277
Epoch 5/300
6343/6343 [==============================] - 7s 1ms/sample - loss: 434.2542 - mse: 434.1494 - val_loss: 426.6633 - val_mse: 426.5600
Epoch 6/300
6343/6343 [==============================] - 6s 1ms/sample - loss: 433.9302 - mse: 433.8278 - val_loss: 428.8991 - val_mse: 428.7982
Epoch 7/300
6343/6343 [==============================] - 6s 1ms/sample - loss: 

2it [04:41, 136.51s/it]

...

3it [06:25, 121.51s/it]

Train on 6344 samples, validate on 1586 samples
Epoch 1/300
6344/6344 [==============================] - 8s 1ms/sample - loss: 517.7599 - mse: 517.6296 - val_loss: 427.4068 - val_mse: 427.2831
Epoch 2/300
6344/6344 [==============================] - 6s 1ms/sample - loss: 446.7891 - mse: 446.6695 - val_loss: 415.9252 - val_mse: 415.8089
Epoch 3/300
6344/6344 [==============================] - 6s 1ms/sample - loss: 437.2683 - mse: 437.1542 - val_loss: 418.7643 - val_mse: 418.6533
Epoch 4/300
6344/6344 [==============================] - 7s 1ms/sample - loss: 431.3867 - mse: 431.2777 - val_loss: 415.4002 - val_mse: 415.2932
Epoch 5/300
6344/6344 [==============================] - 6s 1ms/sample - loss: 435.6006 - mse: 435.4946 - val_loss: 416.3752 - val_mse: 416.2708
Epoch 6/300
6344/6344 [==============================] - 6s 1ms/sample - loss: 434.3161 - mse: 434.2129 - val_loss: 416.4997 - val_mse: 416.3981
Epoch 7/300
6344/6344 [==============================] - 7s 1ms/sample - loss: 435

4it [08:50, 131.02s/it]

Train on 6344 samples, validate on 1586 samples
Epoch 1/300
6344/6344 [==============================] - 8s 1ms/sample - loss: 510.1665 - mse: 510.0350 - val_loss: 441.7712 - val_mse: 441.6458
Epoch 2/300
6344/6344 [==============================] - 7s 1ms/sample - loss: 440.4667 - mse: 440.3451 - val_loss: 435.5436 - val_mse: 435.4250
Epoch 3/300
6344/6344 [==============================] - 6s 1ms/sample - loss: 430.7577 - mse: 430.6415 - val_loss: 467.9852 - val_mse: 467.8706
Epoch 4/300
6344/6344 [==============================] - 6s 1ms/sample - loss: 424.7568 - mse: 424.6434 - val_loss: 479.8296 - val_mse: 479.7179
Epoch 5/300
6344/6344 [==============================] - 7s 1ms/sample - loss: 422.8632 - mse: 422.7529 - val_loss: 437.6054 - val_mse: 437.4962
Epoch 6/300
6344/6344 [==============================] - 7s 1ms/sample - loss: 424.1797 - mse: 424.0716 - val_loss: 435.2100 - val_mse: 435.1026
Epoch 7/300
6344/6344 [==============================] - 6s 1ms/sample - loss: 424

5it [10:52, 130.41s/it]


In [9]:
results

array([20.43285492, 20.17415393, 20.50678873, 20.90181036, 20.95847568])

In [10]:
results.mean()

20.594816722559273